In [1]:
import ee
from tqdm import tqdm

In [2]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWgcX8s6eRAw-fXH39jp1PjZQlcX30xmPxugR0pivyHDleL3E-jJHL8



Successfully saved authorization token.


In [15]:
ee.Initialize()

### Import Sentinel-1 collection and watershed basin shapefile

In [16]:
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
basins = ee.FeatureCollection("users/coreyscher/se-asia_subbasin_01min")

In [17]:
# --------------Runtime parameters ---------------- #

studyArea =  basins.first().geometry()
#basins.filterMetadata('BASINNAME', 'equals', 'GHAASBasin9')


studyStart = '2018-05-01'
studyEnd = '2019-05-01'

monStart = '2019-01-01'
monEnd = '2019-03-01'

lowStart = '2019-06-01'
lowEnd = '2019-07-31'

threshold = 2

vvOwThreshold = -18
vhOwThreshold = -28

connThreshold = 50

#band = ['VV', 'VH']

# Bring in Radar data -----------------------------------------------------------
# ---- Import Sentinel-1 image collection -------
# Filter to get images collected in interferometric wide swath mode. \

s1 = s1.filterBounds(studyArea).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')).filter(ee.Filter.eq('instrumentMode', 'IW'))#.map(gammaMap)

def medianFilt(image):
    im = image.clip(studyArea)
    im = im.focalMedian(15, 'square', 'meters')
    im = im.set('system:time_start', image.get('system:time_start'))
    return im.copyProperties(source = image)


s1 = s1.map(medianFilt)


#----Separate S1 by orbit type ---------------------
s1A = s1.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
s1D = s1.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

# Make list of relative orbit numbers
#

def orbs(image):
    image = ee.Image(image)
    start = image.get('relativeOrbitNumber_start')
    return start

orbitsD = s1D.filterDate(lowStart, lowEnd).toList(s1D.size()).map(orbs).distinct()
print(orbitsD.getInfo())

[136]


In [12]:
#orbitsD = [121,165,19,92] # [106,150,4,48,77,121,19]
# ----------------------------------------------------------------------
# Functions

# Descending track algorithm

def minMax(orbitNumber, pol):
    coll = s1D.filterDate(studyStart,studyEnd).select([pol]).filterMetadata('relativeOrbitNumber_start', 'equals', orbitNumber)
    pol = ee.String(pol)
    min_ = coll.min().rename([pol.cat('_min')])
    #max = coll.filterDate(lowStart, lowEnd).mean().rename([pol.cat('_max')])

    max_ = coll.max().rename([pol.cat('_max')]) #

    mean = coll.mean().rename([pol.cat('_mean')])
    std = coll.reduce(ee.Reducer.stdDev()).rename([pol.cat('_std')]) #.filterDate(lowStart, lowEnd)
    count = coll.count().rename([pol.cat('_count')])
    dif = max_.subtract(min_).rename([pol.cat('_dif')])
    z = max_.subtract(min_).divide(std).rename([pol.cat('_z')])
    return max_.addBands([min_, mean, std, dif, count, z]).set({'pol': pol}).toFloat()





def algDesc(orbitNumber):
    coll = s1D.filterDate(studyStart,studyEnd).filterMetadata('relativeOrbitNumber_start', 'equals', orbitNumber)

    # Calculate relevant statistics for threshold detection 

    vh_stats = minMax(orbitNumber, 'VH')
    vv_stats = minMax(orbitNumber, 'VV')

    vvZMask = vv_stats.select(['VV_z']).gt(threshold)
    vhZMask = vh_stats.select(['VH_z']).gt(threshold)

    #combine the images

    #combined = vh_stats.addBands(vv_stats)

    # classify inundation and masK 

    def openWaterClass(image):
        timeFromStudyStart = image.date().difference(ee.Date(studyStart), 'day')
        date = ee.Image(ee.Number(timeFromStudyStart))
        vv = image.select(['VV']).lt(vvOwThreshold).multiply(date).toInt()
        vh = image.select(['VH']).lt(vhOwThreshold).multiply(date).toInt()


        vvAreaMask = vv.connectedPixelCount(1024, False).gt(connThreshold)
        vhAreaMask = vh.connectedPixelCount(1024, False).gt(connThreshold)

        vv = vv.updateMask(vv.neq(0)).updateMask(vvZMask).updateMask(vvAreaMask)
        vh = vh.updateMask(vh.neq(0)).updateMask(vhZMask).updateMask(vhAreaMask)

        return vh.addBands(vh).copyProperties(image)

    processed = coll.map(openWaterClass)
    onset = processed.reduce(ee.Reducer.firstNonNull()).rename(['VH_first', 'VV_first'])
    offset = processed.reduce(ee.Reducer.lastNonNull()).rename(['VH_last', 'VV_last'])

    return onset.addBands(offset)

  
  #return mid.addBands([min, max]) #max.addBands([min, dif])




descending = orbitsD.map(algDesc)

# mosaic into a single image

descendingMerged = ee.ImageCollection([ee.Image(descending.get(0))]).mosaic() #,
                                          #ee.Image(descending.get(1)),
                                          #ee.Image(descending.get(2))]).mosaic()
                                          
#                                           ee.Image(descending.get(2)),
#                                           ee.Image(descending.get(3)),
#                                           ee.Image(descending.get(4)),
#                                           ee.Image(descending.get(5)),
#                                           ee.Image(descending.get(6)),
#                                           ee.Image(descending.get(7))])


#
# --- Reduce image to a list -- #

#reduced = descendingMerged.select(['VV_first', 'VV_last']).reduceRegion(reducer = ee.Reducer.toList(), scale = 30, geometry = studyArea, maxPixels = 1e13).getInfo()


In [14]:
# --------------Runtime parameters ---------------- # 

threshold = -2

vvOwThreshold = -18
vhOwThreshold = -28

connThreshold = 20

studyStart
studyEnd


# band = ['VV', 'VH']


# ------------------------------------------------------------------------------
# Add interactive pixel explorer to map
panel = ui.Panel()
panel.style().set('width', '400px', 'position', 'bottom-left')

intro = ui.Panel([
  ui.Label({
    value: 'Basin inspector',
    style: {fontSize: '20px', fontWeight: 'bold'}
  }),
  ui.Label('')
])
panel.widgets().set(0,intro)


# Add time series inspector ----------------------------------------------------------------------------
  # panels to hold lon/lat values
  lon = ui.Label()
  lat = ui.Label()
  subBasin = ui.Label()
  
  dates = {
  "2015 - 2016": ["2015-01-01", "2016-01-01"],
  "2016 - 2017": ["2016-01-01", "2017-01-01"],
  "2017 - 2018": ["2017-01-01", "2018-01-01"],
  "2018 - 2019": ["2018-01-01", "2019-0'1-01"],
  "2019 - 2020": ["2019-01-01", "2020-01-01"],
}

select = ui.Select({
  items: Object.keys(dates),
  style: {textAlign: 'center'},
  onChange: function(key) {
    dates = ee.Dictionary(dates)
    studyStart = ee.List(dates.get(key)).get(0)
    studyEnd = ee.List(dates.get(key)).get(1)
    print(studyStart)
  }
})
panel.widgets().set(1, select)
  # Register a callback on the default map to be invoked when the map is clicked
Map.onClick(function(coords) {
  
  # Update the lon/lat panel with values from the click event.
  s1A
  s1D
  studyArea
  descending
  point
  openWater # 25
  wetland # 11
  rice # 17
  
  
  point = ee.Geometry.Point(coords.lon, coords.lat)
  studyArea = ee.Feature(basins.filterBounds(point).first())
  
  lon.setValue('Longitude: ' + coords.lon.toFixed(5)),
  lat.setValue('Latitude: ' + coords.lat.toFixed(5))
  
  subBasin.setValue('CELLID ' + studyArea.get('CELLID'))
  
  # Add a green dot to the map where the user clicked.
  Map.layers().set(2, ui.Map.Layer(studyArea, {color: 'red', name: 'Selected basin'}))
  Map.layers().set(3, ui.Map.Layer(point, {color: '00ff00', name: 'Point selected'}))
 
 # Run the algorithm
 
 # Bring in Radar data -----------------------------------------------------------
  
  # ----Separate S1 by orbit type ---------------------
  s1A = s1.filterBounds(studyArea.geometry())
  # Filter to get images with VV and VH dual polarization.
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
  # Filter to get images collected in interferometric wide swath mode.
      .filter(ee.Filter.eq('instrumentMode', 'IW')).filterDate(studyStart, studyEnd).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
  s1D = s1.filterBounds(studyArea.geometry())
  # Filter to get images with VV and VH dual polarization.
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
  # Filter to get images collected in interferometric wide swath mode.
      .filter(ee.Filter.eq('instrumentMode', 'IW')).filterDate(studyStart, studyEnd).filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
# --------------------------------------------------------------
  # Functions
  
  # Descending track algorithm
  
  medianFilt = function(image){
    return image.focalMedian(15, 'square', 'meters').clip(studyArea).copyProperties(image)
  }
  
  minMax = function(pol){
    coll = s1D.filterDate(studyStart,studyEnd).select([pol])
    filteredColl = coll.map(medianFilt)
    pol = ee.String(pol)
    min = filteredColl.min().rename([pol.cat('_min')])
    # max = coll.filterDate(lowStart, lowEnd).mean().rename([pol.cat('_max')])
    
    max = filteredColl.max().rename([pol.cat('_max')]) # 
    
    mean = filteredColl.mean().rename([pol.cat('_mean')])
    std = filteredColl.reduce(ee.Reducer.stdDev()).rename([pol.cat('_std')]) # .filterDate(lowStart, lowEnd)
    count = filteredColl.count().rename([pol.cat('_count')])
    dif = max.subtract(min).rename([pol.cat('_dif')])
    z = max.subtract(min).divide(std).rename([pol.cat('_z')])
    return max.addBands([min, mean, std, dif, count, z]).set({'pol': pol}).toFloat()
    
  }
    
    # Calculate relevant statistics for threshold detection 
  
  vh_stats = minMax('VH')
  vv_stats = minMax('VV')
  
  vvZMask = vv_stats.select(['VV_z']).gt(2)
  vhZMask = vh_stats.select(['VH_z']).gt(2)
  
  # combine the images
  
  # combined = vh_stats.addBands(vv_stats)
  
  # classify inundation and masK 
  
  openWaterClass = function(image){
    # timeFromStudyStart = image.date().difference(ee.Date(studyStart), 'day')
    # date = ee.Image(ee.Number(timeFromStudyStart))
    # vv = image.select(['VV']).lt(vvOwThreshold)
    vh = image.select(['VH']).lt(vhOwThreshold)
    
    
    # vvAreaMask = vv.connectedPixelCount(1024, false).gt(connThreshold)
    vhAreaMask = vh.connectedPixelCount(1024, false).gt(connThreshold)
    
    # vv = vv.updateMask(vv.neq(0)).updateMask(vvZMask).updateMask(vvAreaMask)
    vh = vh.updateMask(vh.neq(0)).updateMask(vhZMask).updateMask(vhAreaMask)
    
    openWater = vh.updateMask(lulc.eq(25))
    wetland = vh.updateMask(lulc.eq(11))
    rice = vh.updateMask(lulc.eq(17))
    com = ee.Image(openWater.copyProperties(image)).set('system:time_start', image.get('system:time_start')).addBands(wetland).addBands(rice).rename(['openWater', 'wetland', 'rice'])
  
    return com
    
  }
  
  descending = s1D.map(openWaterClass)


 chartStyle = {series: {
    0: {lineWidth: 1, color: 'c4aead', pointSize: 2},
    1: {lineWidth: 1, color: '000000', pointSize: 2}
  },
   title: '"Inundated" pixels',
    vAxis: {title: 'count'},
    hAxis: {title: 'date', format: 'MM-yy', gridlines: {count: 7}}
 }
  
  # Create a chart.
  
  reduced = ui.Chart.image.series(descending, studyArea, ee.Reducer.sum(), 30)
  reduced.setOptions({
    title: 'Inundated pixels',
    vAxis: {title: 'count'},
    hAxis: {title: 'date', format: 'MM-yy', gridlines: {count: 7}},
  }).setOptions(chartStyle)
  
  # panel.widgets().set(1, s1AChart)
  
  panel.widgets().set(2, reduced)
  print(reduced.getOptions())
  # panel.widgets().set(1, s1DChart)
  
})

ui.root.insert(0, panel)

Map.style().set('cursor', 'crosshair')

Map.setOptions('SATELLITE')
Map.centerObject(example_area, 12)




{'type': 'Image',
 'bands': [{'id': 'VH_first',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'VV_first',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'VH_last',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'VV_last',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [ ]:
counts